# Two Sum

In [9]:
class Solution(object):
    def twoSum(self, nums, target):
        seen = {} 
        for i, num in enumerate(nums):
            complement = target - num
            if complement in seen:        
                return [seen[complement], i]
            seen[num] = i
        return []

s = Solution()
print(s.twoSum([2, 7, 11, 15], 9))    # [0, 1]
print(s.twoSum([3, 2, 4], 6))         # [1, 2]
print(s.twoSum([3, 3], 6))            # [0, 1]


[0, 1]
[1, 2]
[0, 1]


### Simplified Financial Application

In [6]:
returns = [0.03, -0.01, -0.03, 0.01, 0.05, -0.05]
tickers = ['AAPL', 'GOOG', 'TSLA', 'AMZN', 'META', 'NFLX']
target_return = 0.00

def neutral_pairs(returns, tickers, target_return, tolerance=1e-6):
    
    # Finds pairs of assets whose returns sum approximately to a target return.
    
    seen = {}
    result = []

    for i, r in enumerate(returns):
        complement = target_return - r

        # Round to avoid floating point precision issues
        complement = round(complement, 6)
        r_rounded = round(r, 6)

        if complement in seen:
            # Store the pair of ticker names
            result.append((tickers[seen[complement]], tickers[i]))
        else:
            seen[r_rounded] = i

    return result

print(neutral_pairs(returns, tickers, target_return))
# Output: [('AAPL', 'TSLA'), ('META', 'NFLX')]



[('AAPL', 'TSLA'), ('GOOG', 'AMZN'), ('META', 'NFLX')]


### Expanded Financial Application

In [7]:
import yfinance as yf
import pandas as pd
import os

# Diversified ticker set
tickers = ['AAPL', 'GOOG', 'TSLA', 'NVDA',  # Tech
           'GE',                           # Industrial
           'JNJ',                          # Healthcare
           'PG',                           # Consumer
           'BHP']                          # Mining

# Save to CSV to avoid re-downloading
data_path = "cached_prices.csv"

if os.path.exists(data_path):
    prices = pd.read_csv(data_path, index_col=0, parse_dates=True)
else:
    prices = yf.download(tickers, start="2018-01-01")['Adj Close']
    prices.to_csv(data_path)

returns = prices.pct_change().dropna()

Failed to get ticker 'AAPL' reason: Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  8 of 8 completed

7 Failed downloads:
['BHP']: Timeout('Failed to perform, curl: (28) Connection timed out after 10004 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['AAPL']: YFTzMissingError('possibly delisted; no timezone found')
['PG']: Timeout('Failed to perform, curl: (28) Operation timed out after 10014 milliseconds with 5815 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['JNJ']: Timeout('Failed to perform, curl: (28) Connection timed out after 10012 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['TSLA']: Timeout('Failed to perform, curl: (28) Operation timed out after 10006 milliseconds with 1959 bytes received. 

In [8]:
def neutral_pairs(returns, tickers, target_return=0.0, tolerance=1e-6):
    seen = {}
    result = []

    for i, r in enumerate(returns):
        complement = round(target_return - r, 6)
        r_rounded = round(r, 6)
        if abs(complement - r_rounded) < tolerance and complement in seen:
            result.append((tickers[seen[complement]], tickers[i]))
        else:
            seen[r_rounded] = i
    return result

# Example (most recent date):
recent = returns.iloc[-1]
print(neutral_pairs(recent.tolist(), returns.columns.tolist(), 0.0))


IndexError: single positional indexer is out-of-bounds

### Full Financial Application

#### rolling covariance weighted hedging tool

In [ ]:
from scipy.stats.mstats import winsorize
import numpy as np

# Winsorize returns to reduce impact of outliers
winsorized_returns = returns.apply(lambda x: pd.Series(winsorize(x, limits=[0.1, 0.1]).data, index=x.index))

# Basic correlation matrix (Pearson)
corr_matrix = winsorized_returns.corr()
vols = winsorized_returns.std()

# Stress definition and filtering
stress_days = (returns < -0.05).any(axis=1)
corr_during_stress = winsorized_returns[stress_days].corr()

def pair_score(a, b, alpha=0.6, beta=0.3, gamma=0.1):
    c1 = corr_matrix.loc[a, b]
    c2 = 1 - corr_during_stress.loc[a, b]
    vol_ratio = abs(vols[a] - vols[b])
    return alpha * c1 + beta * c2 - gamma * vol_ratio

def find_low_corr_pairs(thresh=0.3):
    pairs = []
    for i, a in enumerate(corr_matrix.columns):
        for j in range(i + 1, len(corr_matrix.columns)):
            b = corr_matrix.columns[j]
            c = corr_matrix.loc[a, b]
            if abs(c) < thresh:
                score = pair_score(a, b)
                pairs.append((a, b, c, score))
    return sorted(pairs, key=lambda x: x[3])

# Print top low-correlation pairs
low_corr_pairs = find_low_corr_pairs()
for p in low_corr_pairs:
    print(f"Pair: {p[0]} & {p[1]} | Corr: {p[2]:.3f} | Score: {p[3]:.3f}")


In [3]:
import yfinance as yf
import pandas as pd

# Download historical prices
tickers = ['AAPL', 'GOOG', 'TSLA', 'AMZN', 'META', 'NFLX']
prices = yf.download(tickers, start="2024-06-01", end="2025-06-01")['Adj Close']
returns = prices.pct_change().dropna()

# === TwoSum: Return-neutral pair detection === #
def TwoSum(tickers, returns, target_return=0.0, tolerance=1e-4):
    seen = {}
    result = []
    for i, r in enumerate(returns):
        complement = round(target_return - r, 6)
        r_rounded = round(r, 6)
        if complement in seen:
            result.append((tickers[seen[complement]], tickers[i]))
        else:
            seen[r_rounded] = i
    return result

# === Apply TwoSum to each day's returns === #
def rolling_neutral_pairs(returns_df, target_return=0.0, tolerance=1e-4):
    results = []
    for date, row in returns_df.iterrows():
        result = TwoSum(returns_df.columns.tolist(), row.tolist(), target_return, tolerance)
        results.append((date, result))
    return results


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
import yfinance as yf
import pandas as pd

# Download historical prices
tickers = ['AAPL', 'GOOG', 'TSLA', 'AMZN', 'META', 'NFLX']
prices = yf.download(tickers, start="2024-06-01", end="2025-06-01")['Adj Close']
returns = prices.pct_change().dropna()

# === Compute correlation matrices and volatilities === #
corr_matrix = returns.corr()
vols = returns.std()

# Identify stress periods: days where any asset drops more than 5%
drawdowns = (returns < -0.05).any(axis=1)

# Correlation matrix during stress periods
corr_during_stress = returns[drawdowns].corr()

# === Optional filters === #
def passes_corr_filter(a, b, threshold=0.6):
    return corr_matrix.loc[a, b] > threshold

def stress_inverted_filter(a, b, max_corr=0.2):
    return corr_during_stress.loc[a, b] < max_corr

# === Pair scoring based on normal and stress correlation + volatility divergence === #
def pair_score(a, b, alpha=0.6, beta=0.3, gamma=0.1):
    c1 = corr_matrix.loc[a, b]  # correlation under normal
    c2 = 1 - corr_during_stress.loc[a, b]  # inverted correlation under stress
    vol_ratio = abs(vols[a] - vols[b])
    return alpha * c1 + beta * c2 - gamma * vol_ratio


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# Load prices
tickers = ['AAPL', 'GOOG', 'TSLA', 'AMZN', 'META', 'NFLX']
prices = yf.download(tickers, start="2024-06-01", end="2025-06-01")['Adj Close']
returns = prices.pct_change().dropna()

# Winsorize returns to reduce impact of outliers
winsorized_returns = returns.apply(lambda x: pd.Series(winsorize(x, limits=[0.1, 0.1]).data, index=x.index))

# Basic correlation matrix (Pearson)
corr_matrix = winsorized_returns.corr()

# Define regime classification
regime_matrix = pd.DataFrame(index=corr_matrix.index, columns=corr_matrix.columns)

for a in corr_matrix.columns:
    for b in corr_matrix.columns:
        if a == b:
            regime_matrix.loc[a, b] = "self"
        else:
            corr = corr_matrix.loc[a, b]
            if corr > 0.6:
                regime_matrix.loc[a, b] = "risk-off"
            elif corr < -0.6:
                regime_matrix.loc[a, b] = "unstable"
            else:
                regime_matrix.loc[a, b] = "baseline"

# Optional: rolling correlation over time (not yet used in this static version)
rolling_corrs = winsorized_returns.rolling(21).corr()

# Alternative: Kendall’s tau for non-Gaussian behavior
kendall_corr = winsorized_returns.corr(method='kendall')

# Stress-based weighting
weights = (returns < -0.05).astype(int) * 2 + 1
weighted_corr = winsorized_returns.mul(weights).corr()

# Stress-period-only correlation matrix
stress_periods = returns < -0.05
corr_during_stress = winsorized_returns[stress_periods.any(axis=1)].corr()

# Volatility for each asset
vols = winsorized_returns.std()

# Pair scoring function
def pair_score(a, b, alpha=0.6, beta=0.3, gamma=0.1):
    c1 = corr_matrix.loc[a, b]
    c2 = 1 - corr_during_stress.loc[a, b]
    vol_ratio = abs(vols[a] - vols[b])
    return alpha * c1 + beta * c2 - gamma * vol_ratio

# Find top-scoring uncorrelated pairs
def find_low_corr_pairs(corr_matrix, threshold=0.3):
    pairs = []
    for i, a in enumerate(corr_matrix.columns):
        for j in range(i + 1, len(corr_matrix.columns)):
            b = corr_matrix.columns[j]
            corr = corr_matrix.loc[a, b]
            if abs(corr) < threshold:
                score = pair_score(a, b)
                pairs.append((a, b, corr, score))
    return sorted(pairs, key=lambda x: x[3])

# Final result
low_corr_pairs = find_low_corr_pairs(corr_matrix)
for p in low_corr_pairs:
    print(f"Pair: {p[0]} & {p[1]} | Corr: {p[2]:.3f} | Score: {p[3]:.3f}")

# That regime-tracking logic written out over time?
# A full notebook version with heatmaps?
# A scoring function that inverts weights during drawdowns (i.e., reward negative correlation when others go up)?